In [1]:
import numpy as np
from sympy import *
from sympy.solvers import solve
init_printing(use_unicode=True)

In [2]:
def simulate_M(seed: int, p: int) -> Matrix:
    np.random.seed(seed=seed)
    
    normal_matrix = np.random.normal(0, 1, (p, p))
    for i in range(p): # adjust diagonal entries s.t. M stable
        normal_matrix[i, i] = - np.sum(np.abs(normal_matrix[i, :]))

    return Matrix(normal_matrix)

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def solve_lyapunov_cov(M: Matrix, C: Matrix, p: int) -> Matrix:
    Sigma = MatrixSymbol("Sigma", p, p).as_explicit()
    Sigma_solved = solve(M*Sigma + Sigma*M.T + C, Sigma)
    scheme = [[Sigma_solved[Sigma[i,j]] for i in range(p)] for j in range(p)]
    Sigma_new = Matrix(scheme)
    return Sigma_new

def create_A_Sigma(cov: Matrix, p: int) -> Matrix:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = MatrixSymbol("A", row_num, col_num).as_mutable()
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Matrix, mean: Matrix, p: int) -> Matrix:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = Matrix.hstack(identity, mean[i] * eye(p))
    return Matrix.vstack(A_Sigma, identity)

In [9]:
num_exp = 1000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix

m11, m12, m13 = symbols('m11'), symbols('m12'), symbols('m13')
m21, m22, m23 = symbols('m21'), symbols('m22'), symbols('m23')
m31, m32, m33 = symbols('m31'), symbols('m32'), symbols('m33')

index_intervention = 0
b = 2

Universally:
* save estimates and their Frobenius distances to the true $M^*$
* $\texttt{frob = -1}$ if no result could be produced

## 2 nodes

In [54]:
p = 2

M = Matrix([[m11, m12], [m21, m22]])
C = 2 * eye(p)
vec_M = Matrix([[M[j, i]] for i in range(p) for j in range(p)])
vech_C = Matrix([[C[i, j]] for i in range(p) for j in range(i, p)])
unit_v = eye(p).col(index_intervention)

reg_param = [0, 0.01, 0.1]
num_param = len(reg_param)

### $\texttt{index\_deletion = 0}$

In [ ]:
index_deletion = 0
offset = 1

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, 1 + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

4


In [ ]:
index_deletion = 0
offset = 1

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### $\texttt{index\_deletion = 1}$

In [ ]:
index_deletion = 1
offset = 12

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### $\texttt{index\_deletion = 2}$

In [ ]:
index_deletion = 2
offset = 123

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### $\texttt{index\_deletion = 3}$

In [ ]:
index_deletion = 3
offset = 1234

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### $\texttt{index\_deletion = 4}$

In [ ]:
index_deletion = 4
offset = 12345

c = Matrix.vstack(vech_C, - b * unit_v)
c.row_del(index_deletion)

estimate_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

## 3 nodes

In [10]:
p = 3

M = Matrix([[m11, m12, m13], [m21, m22, m23], [m31, m32, m33]])
C = 2 * eye(p)
vec_M = Matrix([[M[j, i]] for i in range(p) for j in range(p)])
vech_C = Matrix([[C[i, j]] for i in range(p) for j in range(i, p)])
unit_v = eye(p).col(index_intervention)
c = Matrix.vstack(vech_C, - b * unit_v)

reg_param = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
num_param = len(reg_param)

### All entries

In [ ]:
offset = 120

estimate_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### One random entry missing

In [23]:
entries = [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]
offset = 1208

estimate_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)
    np.random.seed(seed)
    true_M[entries[np.random.randint(len(entries))]] = 0

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

Matrix([[-2.76427352957677, -0.611756413650075, -0.528171752263456], [-1.07296862215617, -4.23991494836113, -2.30153869688028], [1.74481176421648, 0, -2.82505776116868]])
Matrix([[-2.60922077030025, 0, -2.13619609566845], [1.64027080840499, -4.27545375925606, -0.841747365656204], [0.502881417158043, -1.24528808660723, -2.80612172262761]])
Matrix([[-2.32163579201432, 0.436509850511989, 0.0964974680720086], [0, -2.49563988514788, -0.354758979268987], [-0.0827414814824598, -0.627000676823847, -0.753560327282235]])


### Two entries missing I

In [ ]:
offset = 12089

estimate_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)
    true_M[0, 1] = 0
    true_M[2, 0] = 0

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)

### Two entries missing II

In [ ]:
offset = 120899

estimate_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)
    true_M[0, 1] = 0
    true_M[0, 2] = 0

    true_cov = matrix2numpy(solve_lyapunov_cov(M=true_M, C=C, p=p)).astype(float)
    true_mean = (matrix2numpy(b * true_M.inv() * unit_v).astype(float)).flatten()

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    emp_A.row_del(index_deletion)
    # solution = solve(emp_A * vec_M + c, vec_M)